In [198]:
import numpy as np
from sklearn.base import BaseEstimator

def entropy(y):  
    """
    Computes entropy of the provided distribution. Use log(value + eps) for numerical stability
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, n_classes)
        One-hot representation of class labels for corresponding subset
    
    Returns
    -------
    float
        Entropy of the provided subset
    """
    EPS = 0.0005

    # YOUR CODE HERE
    probas = np.mean(y, axis=0)
    log_probas = np.log(probas + EPS)
    
    return -np.sum(probas * log_probas)
    
def gini(y):
    """
    Computes the Gini impurity of the provided distribution
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, n_classes)
        One-hot representation of class labels for corresponding subset
    
    Returns
    -------
    float
        Gini impurity of the provided subset
    """

    # YOUR CODE HERE
    probas = np.mean(y, axis=0)
    return 1 - np.sum(probas ** 2)
    
def variance(y):
    """
    Computes the variance the provided target values subset
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, 1)
        Target values vector
    
    Returns
    -------
    float
        Variance of the provided target vector
    """
    
    # YOUR CODE HERE
    return np.sum((y - np.mean(y)) ** 2) / len(y)

def mad_median(y):
    """
    Computes the mean absolute deviation from the median in the
    provided target values subset
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, 1)
        Target values vector
    
    Returns
    -------
    float
        Mean absolute deviation from the median in the provided vector
    """

    # YOUR CODE HERE
    return np.sum(abs(y - np.median(y))) / len(y)

def one_hot_encode(n_classes, y):
    y_one_hot = np.zeros((len(y), n_classes), dtype=float)
    y_one_hot[np.arange(len(y)), y.astype(int)[:, 0]] = 1.
    return y_one_hot


def one_hot_decode(y_one_hot):
    return y_one_hot.argmax(axis=1)[:, None]


class Node:
    """
    This class is provided "as is" and it is not mandatory to it use in your code.
    """
    def __init__(self, feature_index, threshold, depth=0):
        self.feature_index = feature_index
        self.value = threshold
        self.proba = None
        self.depth = depth
        self.left_child = None
        self.right_child = None
        self.isleaf = False
        
        
class DecisionTree(BaseEstimator):
    all_criterions = {
        'gini': (gini, True), # (criterion, classification flag)
        'entropy': (entropy, True),
        'variance': (variance, False),
        'mad_median': (mad_median, False)
    }

    def __init__(self, n_classes=None, max_depth=np.inf, min_samples_split=2, 
                 criterion_name='gini', debug=False):

        assert criterion_name in self.all_criterions.keys(), 'Criterion name must be on of the following: {}'.format(self.all_criterions.keys())
        
        self.n_classes = n_classes
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

        self.criterion_name = criterion_name
        self.answer = 0

        self.depth = 0
        self.leaves = 0
        self.root = None # Use the Node class to initialize it later
        self.debug = debug

    def make_split(self, feature_index, threshold, X_subset, y_subset):
        """
        Makes split of the provided data subset and target values using provided feature and threshold
        
        Parameters
        ----------
        feature_index : int
            Index of feature to make split with

        threshold : float
            Threshold value to perform split

        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset

        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels for corresponding subset
        
        Returns
        -------
        (X_left, y_left) : tuple of np.arrays of same type as input X_subset and y_subset
            Part of the providev subset where selected feature x^j < threshold
        (X_right, y_right) : tuple of np.arrays of same type as input X_subset and y_subset
            Part of the providev subset where selected feature x^j >= threshold
        """

        # YOUR CODE HERE
        mask = X_subset[:, feature_index] < threshold
        opposite_mask = X_subset[:, feature_index] >= threshold

        y_left = y_subset[mask]
        y_right = y_subset[opposite_mask]
        
        X_left = X_subset[mask]
        X_right = X_subset[opposite_mask]
        
        return (X_left, y_left), (X_right, y_right)
    
    def make_split_only_y(self, feature_index, threshold, X_subset, y_subset):
        """
        Split only target values into two subsets with specified feature and threshold
        
        Parameters
        ----------
        feature_index : int
            Index of feature to make split with

        threshold : float
            Threshold value to perform split

        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset

        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels for corresponding subset
        
        Returns
        -------
        y_left : np.array of type float with shape (n_objects_left, n_classes) in classification 
                   (n_objects, 1) in regression 
            Part of the provided subset where selected feature x^j < threshold

        y_right : np.array of type float with shape (n_objects_right, n_classes) in classification 
                   (n_objects, 1) in regression 
            Part of the provided subset where selected feature x^j >= threshold
        """

        # YOUR CODE HERE
        mask = X_subset[:, feature_index] < threshold
        opposite_mask = X_subset[:, feature_index] >= threshold
        y_left = y_subset[mask]
        y_right = y_subset[opposite_mask]
        return y_left, y_right
        

    def choose_best_split(self, X_subset, y_subset):
        """
        Greedily select the best feature and best threshold w.r.t. selected criterion
        
        Parameters
        ----------
        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset
        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels or target values for corresponding subset
        
        Returns
        -------
        feature_index : int
            Index of feature to make split with
        threshold : float
            Threshold value to perform split
        """
        # YOUR CODE HERE
        G = -np.inf
        feature_index = 0
        threshold = 0
        n_objects, n_features = X_subset.shape
        
        for cur_feature in np.arange(n_features):
            tresh_list = np.unique(X_subset[:, cur_feature])
            for cur_treshold in tresh_list:
                y_left, y_right = self.make_split_only_y(cur_feature, cur_treshold, X_subset, y_subset)
                
                if y_left.shape[0] == 0 or y_right.shape[0] == 0:
                    continue
                    
                L = y_left.shape[0]
                R = y_right.shape[0]
                Q = y_subset.shape[0]                
                cur_G = self.all_criterions[self.criterion_name][0](y_subset) - (self.all_criterions[self.criterion_name][0](y_left) * L / Q) - (self.all_criterions[self.criterion_name][0](y_right) * R / Q)
                
                if cur_G > G:
                    G = cur_G
                    feature_index = cur_feature
                    threshold = cur_treshold                           
        
        return feature_index, threshold
    
    def make_tree(self, X_subset, y_subset, depth = 0):
        """
        Recursively builds the tree
        
        Parameters
        ----------
        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset
        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels or target values for corresponding subset
        
        Returns
        -------
        root_node : Node class instance
            Node of the root of the fitted tree
        """
        leaves = X_subset.shape[0]        
        
        if depth > self.depth:
            self.depth = depth
        
        if leaves > self.min_samples_split and depth < self.max_depth:
            feature, tresh = self.choose_best_split(X_subset, y_subset)
            new_node = Node(feature, tresh, depth)
            
            left, right = self.make_split(feature, tresh, X_subset, y_subset)
                    
            new_node.left_child = self.make_tree(left[0], left[1], depth + 1)
            new_node.right_child = self.make_tree(right[0], right[1], depth + 1)

        else:
            new_node = Node(0, 0, depth=depth)
            if self.classification:
                new_node.proba = np.mean(y_subset, axis=0)
            else:
                new_node.value = np.mean(y_subset)
                       
        return new_node
        
    def fit(self, X, y):
        """
        Fit the model from scratch using the provided data
        
        Parameters
        ----------
        X : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the data to train on
        y : np.array of type int with shape (n_objects, 1) in classification 
                   of type float with shape (n_objects, 1) in regression 
            Column vector of class labels in classification or target values in regression
        
        """
        assert len(y.shape) == 2 and len(y) == len(X), 'Wrong y shape'
        self.criterion, self.classification = self.all_criterions[self.criterion_name]
        if self.classification:
            if self.n_classes is None:
                self.n_classes = len(np.unique(y))
            y = one_hot_encode(self.n_classes, y)

        self.root = self.make_tree(X, y, 1)
        
    def climb_down(self, X, node):
        if node.left_child == None or node.right_child == None:
            if self.classification:
                self.answer = np.argmax(node.proba)
            else:
                self.answer = node.value
        else:
            if X[node.feature_index] < node.value:
                self.climb_down(X, node.left_child)
            else:
                self.climb_down(X, node.right_child)
                    
    
    def predict(self, X):
        """
        Predict the target value or class label  the model from scratch using the provided data
        
        Parameters
        ----------
        X : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the data the predictions should be provided for
        Returns
        -------
        y_predicted : np.array of type int with shape (n_objects, 1) in classification 
                   (n_objects, 1) in regression 
            Column vector of class labels in classification or target values in regression
        
        """

        # YOUR CODE HERE
        n_objects = X.shape[0]
        y_predicted = []
        
        for i in np.arange(n_objects):
            self.climb_down(X[i], self.root)
            y_predicted.append(self.answer)
            
        y_predicted = np.array(y_predicted)
        return y_predicted
    
    def climb_down_proba(self, X, node):
        if node.left_child == None or node.right_child == None:
            self.answer = node.proba
            
        else:
            if X[node.feature_index] < node.value:
                self.climb_down_proba(X, node.left_child)
            else:
                self.climb_down_proba(X, node.right_child)
        
    def predict_proba(self, X):
        """
        Only for classification
        Predict the class probabilities using the provided data
        
        Parameters
        ----------
        X : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the data the predictions should be provided for
        Returns
        -------
        y_predicted_probs : np.array of type float with shape (n_objects, n_classes)
            Probabilities of each class for the provided objects
        
        """
        assert self.classification, 'Available only for classification problem'

        # YOUR CODE HERE
        n_objects = X.shape[0]
        y_predicted_probs = []
        
        for i in np.arange(n_objects):
            self.climb_down_proba(X[i], self.root)
            y_predicted_probs.append(self.answer)              
            
        y_predicted_probs = np.array(y_predicted_probs)
        return y_predicted_probs